# Wavelength calibration

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### Loading crispy's dependencies

### Generate monochromatic wavelengths

In [2]:
# Suppose we create a folder in crispy/ReferenceFiles/ called wavecalR50_770_test
# par.wavecalDir = par.prefix+'/wavecalR50_660/'
par.wavecalDir = par.prefix+'/wavecalR50_770/'
# construct monochromatic wavelengths, doesn't really matter which wavelengths
lamlist = np.arange(700,845,10)


In [3]:
# construct images for each wavelength
par.savePoly = False
from crispy.IFS import createWavecalFiles
createWavecalFiles(par,lamlist)

crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/ReferenceFiles/wavecalR50_770/det_710.fits
crispy - INFO - The number of input pixels per lenslet is 4.285714
crispy - INFO - Using PSFlet gaussian approximation
crispy - WARNING - Assuming slices are evenly spread in wavelengths
crispy - INFO - Done.
crispy - INFO - Performance: 4 seconds total
crispy - INFO - Writing data to ../../../crispy/Referenc

### Loading wavelengths/filelists

In [4]:
#par.lamlist = np.loadtxt(par.wavecalDir + "lamsol.dat")[:, 0]
par.lamlist=lamlist
par.filelist = np.sort(glob.glob(par.wavecalDir + "det*.fits"))

### Run calibration. Choose parameters wisely.

In [5]:
from crispy.tools.wavecal import buildcalibrations
buildcalibrations(par,
                  order=3, # Order of polynomial map of the PSFLets (between 1 and 5)
                  upsample=10, # Upsampling factor of the PSFLets, needs to be at least 5
                  nsubarr=1, # How many zones on the detector have their own PSFLets? 
                  genwavelengthsol=True, # Compute wavelength at the center of all pixels (need to do this at least once)
                  makehiresPSFlets=True, # Compute hires PSFLets (usually False)
                  makePolychrome=True,
                  apodize=False,
                  parallel=True) # Use all computer cores

crispy - INFO - Building calibration files, placing results in ../../../crispy/ReferenceFiles/wavecalR50_770/
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Initializing PSFlet location transformation coefficients
crispy - INFO - Performing initial optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../crispy/ReferenceFiles/wavecalR50_770/det_700.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770/det_710.fits
crispy - INFO - Initializing transformation coefficients with previous values
crispy - INFO - Performing final optimization of PSFlet location transformation coefficients for frame ../../../cris

In [6]:
from crispy.tools.reduction import calculateWaveList
lam_midpts,lam_endpts = calculateWaveList(par,method='lstsq')
print('Wavelengths at midpoints for lstsq (nm): ',lam_midpts)
print('Wavelengths at endpoints for lstsq (nm): ',lam_endpts)
lam_midpts,lam_endpts = calculateWaveList(par,method='optext')
print('Wavelengths at midpoints for optext (nm): ',lam_midpts)
print('Wavelengths at endpoints for optext (nm): ',lam_endpts)

crispy - INFO - Reduced cube will have 10 wavelength bins
('Wavelengths at midpoints for lstsq (nm): ', array([ 707.,  721.,  735.,  749.,  763.,  777.,  791.,  805.,  819.,  833.]))
('Wavelengths at endpoints for lstsq (nm): ', array([ 700.,  714.,  728.,  742.,  756.,  770.,  784.,  798.,  812.,
        826.,  840.]))
crispy - INFO - Reduced cube will have 19 wavelength bins
('Wavelengths at midpoints for optext (nm): ', array([ 703.68421053,  711.05263158,  718.42105263,  725.78947368,
        733.15789474,  740.52631579,  747.89473684,  755.26315789,
        762.63157895,  770.        ,  777.36842105,  784.73684211,
        792.10526316,  799.47368421,  806.84210526,  814.21052632,
        821.57894737,  828.94736842,  836.31578947]))
('Wavelengths at endpoints for optext (nm): ', array([ 700.        ,  707.36842105,  714.73684211,  722.10526316,
        729.47368421,  736.84210526,  744.21052632,  751.57894737,
        758.94736842,  766.31578947,  773.68421053,  781.05263158,
   

In [7]:
print len(par.lamlist)

15
